In [1]:
from datasets import load_dataset

#可用的数据集:https://huggingface.co/datasets

#在线加载一个数据集
dataset = load_dataset(path='lansinuote/ChnSentiCorp', name=None, split=None)

dataset, dataset['train'][0]

c:\Users\Administrator\anaconda3\envs\nlplearn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(DatasetDict({
     train: Dataset({
         features: ['text', 'label'],
         num_rows: 9600
     })
     validation: Dataset({
         features: ['text', 'label'],
         num_rows: 1200
     })
     test: Dataset({
         features: ['text', 'label'],
         num_rows: 1200
     })
 }),
 {'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般',
  'label': 1})

In [2]:
#保存到本地

dataset.save_to_disk('dataset/lansinuote/ChnSentiCorp')

Saving the dataset (1/1 shards): 100%|██████████| 1200/1200 [00:00<00:00, 601837.24 examples/s]


In [3]:
from datasets import load_from_disk

#从本地加载
dataset = load_from_disk('dataset/lansinuote/ChnSentiCorp')

dataset, dataset['train'][0]

(DatasetDict({
     train: Dataset({
         features: ['text', 'label'],
         num_rows: 9600
     })
     validation: Dataset({
         features: ['text', 'label'],
         num_rows: 1200
     })
     test: Dataset({
         features: ['text', 'label'],
         num_rows: 1200
     })
 }),
 {'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般',
  'label': 1})

In [6]:
dataset = load_from_disk('dataset/lansinuote/ChnSentiCorp')


#遍历每一条数据并进行处理
def f(data):
    #新增字段
    data['new_column'] = data['text'][:5]

    #删除字段
    del data['label']

    #修改字段
    data['text'] = 'prefix_' + data['text']

    return data


dataset = dataset.map(f)

dataset, dataset['train'][0]

(DatasetDict({
     train: Dataset({
         features: ['text', 'new_column'],
         num_rows: 9600
     })
     validation: Dataset({
         features: ['text', 'new_column'],
         num_rows: 1200
     })
     test: Dataset({
         features: ['text', 'new_column'],
         num_rows: 1200
     })
 }),
 {'text': 'prefix_选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般',
  'new_column': '选择珠江花'})

## 过滤datasets

In [5]:
dataset = load_from_disk('dataset/lansinuote/ChnSentiCorp')

#过滤数据集
f = lambda x: x['label'] == 1

dataset = dataset.filter(f)

dataset, dataset['train'][0]

Filter: 100%|██████████| 1200/1200 [00:00<00:00, 96252.98 examples/s]


(DatasetDict({
     train: Dataset({
         features: ['text', 'label'],
         num_rows: 4799
     })
     validation: Dataset({
         features: ['text', 'label'],
         num_rows: 593
     })
     test: Dataset({
         features: ['text', 'label'],
         num_rows: 608
     })
 }),
 {'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般',
  'label': 1})

## Multiple threads data processing

- Multi threaded parallel
- 可应用在大量数据集场景，其效率优胜 `map()`

In [7]:
dataset = load_from_disk('dataset/lansinuote/ChnSentiCorp')


#map和filter时可以使用批量处理,同时使用多线程并行处理
#在某些环境下使用多线程会卡住,出现这种情况时请切换到单线程,num_proc=1
def f(data):
    data['new_column'] = [i[:5] for i in data['text']]
    return data

# "Batch"指的是一次训练或推理中处理的一组样本。
dataset = dataset.map(f, batched=True, batch_size=5, num_proc=2)

dataset, dataset['train'][0]

Map (num_proc=2): 100%|██████████| 1200/1200 [00:01<00:00, 656.52 examples/s]


(DatasetDict({
     train: Dataset({
         features: ['text', 'label', 'new_column'],
         num_rows: 9600
     })
     validation: Dataset({
         features: ['text', 'label', 'new_column'],
         num_rows: 1200
     })
     test: Dataset({
         features: ['text', 'label', 'new_column'],
         num_rows: 1200
     })
 }),
 {'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般',
  'label': 1,
  'new_column': '选择珠江花'})

## Delete String

In [8]:
dataset = load_from_disk('dataset/lansinuote/ChnSentiCorp')

#删除字段
dataset = dataset.remove_columns(['label'])

dataset, dataset['train'][0]

(DatasetDict({
     train: Dataset({
         features: ['text'],
         num_rows: 9600
     })
     validation: Dataset({
         features: ['text'],
         num_rows: 1200
     })
     test: Dataset({
         features: ['text'],
         num_rows: 1200
     })
 }),
 {'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般'})

## Rename String

In [9]:
dataset = load_from_disk('dataset/lansinuote/ChnSentiCorp')

#重命名字段
dataset = dataset.rename_columns({'text': 'new_text', 'label': 'new_label'})

dataset, dataset['train'][0]

(DatasetDict({
     train: Dataset({
         features: ['new_text', 'new_label'],
         num_rows: 9600
     })
     validation: Dataset({
         features: ['new_text', 'new_label'],
         num_rows: 1200
     })
     test: Dataset({
         features: ['new_text', 'new_label'],
         num_rows: 1200
     })
 }),
 {'new_text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般',
  'new_label': 1})

## Set data type

In [10]:
dataset = load_from_disk('dataset/lansinuote/ChnSentiCorp')

#设置字段的数据类型,可选的有np,pt,tf
dataset.set_format('pt', columns=['label'], output_all_columns=True)

dataset, dataset['train'][0]

(DatasetDict({
     train: Dataset({
         features: ['text', 'label'],
         num_rows: 9600
     })
     validation: Dataset({
         features: ['text', 'label'],
         num_rows: 1200
     })
     test: Dataset({
         features: ['text', 'label'],
         num_rows: 1200
     })
 }),
 {'label': tensor(1),
  'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般'})

## Merge datasets

In [11]:
from datasets import concatenate_datasets

dataset = load_from_disk('dataset/lansinuote/ChnSentiCorp')

#合并多个数据集
dataset = concatenate_datasets(list(dataset.values()))

dataset, dataset[0]

(Dataset({
     features: ['text', 'label'],
     num_rows: 12000
 }),
 {'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般',
  'label': 1})

## Split datasets

In [12]:
dataset = load_from_disk('dataset/lansinuote/ChnSentiCorp')

#切分一个数据集为训练集和测试机,可以指定比例,也可以直接指定数量
dataset = dataset['train'].train_test_split(test_size=0.1, train_size=8640)

dataset, dataset['train'][0]

(DatasetDict({
     train: Dataset({
         features: ['text', 'label'],
         num_rows: 8640
     })
     test: Dataset({
         features: ['text', 'label'],
         num_rows: 960
     })
 }),
 {'text': '不算是一本很专业的书，但作为日常养生的参考还是不错，有很多从前不知道的保养知识。看了此书对中医养生有了更浓厚的兴趣，当然尤其是能使自己又美丽又健康的法子，正在尝试，但愿有效果。呵呵，现在买了很多花草茶，还有茯苓粉，打算再做种子眼霜。嘻嘻，女人就是爱折腾，只要能让自己变美，就绝不怕麻烦，所以，从这个方面讲，算是本好书吧',
  'label': 1})

## Split data subsets

In [13]:
dataset = load_from_disk('dataset/lansinuote/ChnSentiCorp')

#取数据集中的某些数据
#[5, 15, 20, 50]表示取第5,15,20,50条数据
dataset = dataset['train'].select([5, 15, 20, 50])

dataset, dataset[0]

(Dataset({
     features: ['text', 'label'],
     num_rows: 4
 }),
 {'text': '机器背面似乎被撕了张什么标签，残胶还在。但是又看不出是什么标签不见了，该有的都在，怪', 'label': 0})